<a href="https://colab.research.google.com/github/jiwon-hae/LLama-training/blob/main/(Part_4)DPO_model_evaluation_using_vLLM(template).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 필요 Library install
!pip install transformers==4.40.0 peft==0.10.0 trl==0.8.6 bitsandbytes accelerate==0.29.0 vllm==0.4.3 gradio==4.29.0 pydantic==2.5.2

In [2]:
!pip install flash-attn==2.5.5 --no-build-isolation

In [3]:
# HF token 설정
from huggingface_hub import notebook_login
notebook_login()

In [4]:
# Google Drive Import
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls /content/drive/MyDrive/fine_tune_output

checkpoint-100	checkpoint-50  runs


In [6]:
!ls /content/drive/MyDrive/dpo_output_1

checkpoint-10  checkpoint-20  checkpoint-30  checkpoint-40  runs


In [7]:

!ls /content/drive/MyDrive/dpo_output_2

checkpoint-10  checkpoint-20  checkpoint-30  checkpoint-40  runs


In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
from vllm import LLM, SamplingParams
import gradio as gr

from vllm import LLM, SamplingParams

vllm_model = LLM(
    model = "Evion/llama3-alpaca-tuned-and-merged",
    tokenizer = "Evion/llama3-alpaca-tuned-and-merged",
    gpu_memory_utilization=0.85,
    enable_lora = True
)

sampling_params = SamplingParams(temperature=0.05, top_p=0.95, max_tokens=256)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

INFO 02-02 07:05:52 llm_engine.py:161] Initializing an LLM engine (v0.4.3) with config: model='Evion/llama3-alpaca-tuned-and-merged', speculative_config=None, tokenizer='Evion/llama3-alpaca-tuned-and-merged', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=Evion/llama3-alpaca-tuned-and-merged)


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

INFO 02-02 07:07:59 model_runner.py:146] Loading model weights took 14.9634 GB
INFO 02-02 07:08:01 gpu_executor.py:83] # GPU blocks: 8585, # CPU blocks: 2048
INFO 02-02 07:08:03 model_runner.py:854] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 02-02 07:08:03 model_runner.py:858] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 02-02 07:08:11 model_runner.py:924] Graph capturing finished in 8 secs.


In [9]:
dpo_output_1_base_path = "/content/drive/MyDrive/dpo_output_1/"
dpo_output_2_base_path = "/content/drive/MyDrive/dpo_output_2/"

lora_configs = {
    "dpo_output_1_cpk_10": (1, dpo_output_1_base_path + "checkpoint_10"),
    "dpo_output_1_cpk_20": (2, dpo_output_1_base_path + "checkpoint_20"),
    "dpo_output_1_cpk_30": (3, dpo_output_1_base_path + "checkpoint_30"),
    "dpo_output_2_cpk_10": (5, dpo_output_2_base_path + "checkpoint_10"),
    "dpo_output_2_cpk_20": (6, dpo_output_2_base_path + "checkpoint_20"),
    "dpo_output_2_cpk_30": (7, dpo_output_2_base_path + "checkpoint_30"),
    "dpo_output_2_cpk_40": (8, dpo_output_2_base_path + "checkpoint_40")
}

In [12]:
from vllm.lora.request import LoRARequest

dpo_output_1_base_path = "/content/drive/MyDrive/dpo_output_1/"
dpo_output_2_base_path = "/content/drive/MyDrive/dpo_output_2/"

lora_configs = {
    "dpo_output_1_cpk_10": (1, dpo_output_1_base_path + "checkpoint-10"),
    "dpo_output_1_cpk_20": (2, dpo_output_1_base_path + "checkpoint-20"),
    "dpo_output_1_cpk_30": (3, dpo_output_1_base_path + "checkpoint-30"),
    "dpo_output_2_cpk_10": (5, dpo_output_2_base_path + "checkpoint-10"),
    "dpo_output_2_cpk_20": (6, dpo_output_2_base_path + "checkpoint-20"),
    "dpo_output_2_cpk_30": (7, dpo_output_2_base_path + "checkpoint-30")
}

def generate_text(raw_input, temperature = 0.05, top_p = 0.95, max_tokens=256, lora_mode = "default"):
    sampling_params = SamplingParams(
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens
    )
    alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

    ### Instruction:
    {}

    ### Response:
    {}"""
    prompt = alpaca_prompt.format(raw_input,"")
    lora_config = {}



    if lora_mode != "default":
        lora_config["lora_request"] = LoRARequest(lora_mode, lora_configs[lora_mode][0], lora_configs[lora_mode][1])

    outputs = vllm_model.generate(
        [prompt],
        sampling_params,
        **lora_config,
    )
    return outputs[0].outputs[0].text

In [13]:
questions = [
    "List three ways to reduce plastic waste in daily life.",
    "Write a haiku about artificial intelligence",
    "How can I improve my public speaking skills?",
    "AI 분야에서 사용하는 LLM이라는 용어가 뭔지 설명해줘",
    "What is a famous tall tower in Paris?",
    "What is Fine-Tuning?",
    "15와 25의 최소공배수를 구하시오.",
    "제2차 세계대전의 주요 원인은 무엇이었나요?",
    "광합성 과정을 간단히 설명해주세요.",
    "셰익스피어의 '햄릿'에서 주인공의 성격을 분석해보세요.",
    "기후 변화가 전 세계적으로 미치는 영향에 대해 설명해주세요.",
    "파이썬에서 리스트와 튜플의 차이점은 무엇인가요?",
    "인공지능은 의식을 가질 수 있을까요? 그 이유는 무엇인가요?",
    "'안녕하세요, 오늘 날씨가 좋네요'를 영어로 번역해주세요.",
    "우주 여행을 주제로 한 짧은 이야기를 만들어보세요.",
    "인플레이션이 경제에 미치는 영향을 설명해주세요."
]

In [ ]:
print("Default model result -----")
print(generate_text(raw_input = questions[0], lora_mode = "default"))
for mode in lora_configs.keys():
  print(f"{mode} model result ------")
  print(generate_text(raw_input = questions[0], lora_mode = mode))
  print()

Default model result -----


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.52s/it, Generation Speed: 60.52 toks/s]


 1. Reduce the use of single-use plastic items such as straws, plastic bags, and disposable cutlery.
     2. Use reusable containers and bags for food and shopping.
     3. Support businesses that use sustainable packaging and products.
     4. Recycle plastic items properly and avoid littering.
     5. Educate others about the impact of plastic waste and encourage them to make changes in their own lives.    
dpo_output_1_cpk_10 model result ------


Processed prompts: 100%|██████████| 1/1 [00:08<00:00,  8.48s/it, Generation Speed: 6.37 toks/s]


 1. Reduce the use of single-use plastic items such as straws, plastic bags, and disposable cutlery.
     2. Use reusable containers and bags for food and shopping.
     3. Recycle plastic items properly and avoid littering.

dpo_output_1_cpk_20 model result ------


...

Processed prompts: 100%|██████████| 1/1 [00:09<00:00,  9.33s/it, Generation Speed: 5.68 toks/s]


 - Reduce the use of single-use plastic items such as plastic bags, straws, and cutlery.
     - Reuse plastic containers and bottles by washing and reusing them.
     - Recycle plastic items by placing them in designated bins for recycling.

dpo_output_2_cpk_10 model result ------


Processed prompts: 100%|██████████| 1/1 [00:19<00:00, 19.66s/it, Generation Speed: 10.33 toks/s]


 1. Reduce the use of single-use plastic items such as straws, plastic bags, and disposable cutlery.
     2. Use reusable containers, bags, and water bottles instead of single-use plastic ones.
     3. Support businesses that use eco-friendly packaging and products.
     4. Recycle plastic items whenever possible.
     5. Avoid buying products that contain excessive plastic packaging.
     6. Support initiatives that promote the reduction of plastic waste, such as beach clean-ups and recycling programs.
     7. Educate yourself and others about the impact of plastic waste on the environment and how to reduce it.
     8. Support the development of alternative materials to plastic, such as biodegradable and compostable products.
     9. Avoid purchasing products that contain microplastics, such as some cosmetics and cleaning products.
     10. Support the development of new technologies that can help reduce plastic waste, such as biodegradable plastics and recycling technologies.    

dp

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, Generation Speed: 0.00 toks/s]

In [ ]:
for idx, question in enumerate(questions):
  print(f"Question_{idx}: {question}")
  print("Default model result -----")
  print(generate_text(raw_input = question, lora_mode = "default"))
  for mode in lora_configs.keys():
    print(f"{mode} model result ------")
    print(generate_text(raw_input = question, lora_mode = mode))
    print()

Question_0: List three ways to reduce plastic waste in daily life.
Default model result -----


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.07it/s, Generation Speed: 58.09 toks/s]


 1. Reduce the use of single-use plastic items such as straws, plastic bags, and disposable cutlery.
     2. Use reusable containers and bags for food and shopping.
     3. Recycle plastic items properly and avoid littering.
dpo_output_1_cpk_10 model result ------


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.56s/it, Generation Speed: 50.12 toks/s]


 - Reduce the use of single-use plastic items such as straws, plastic bags, and disposable cutlery.
     - Use reusable containers and bags for food and shopping.
     - Support businesses that use eco-friendly packaging and products.
     - Recycle plastic items whenever possible.
     - Avoid buying products with excessive packaging.
     - Support initiatives to reduce plastic waste in your community.

dpo_output_1_cpk_20 model result ------


...

 인플레이션은 경제에서 발생하는 물가 상승을 의미합니다. 인플레이션은 경제의 여러 가지 측면에 영향을 미칠 수 있습니다. 인플레이션은 소비자들이 물가를 더 높게 인식하게 하여 소비를 줄이는 데 영향을 미칠 수 있습니다. 이는 기업들이 더 많은 수익을 내기 위해 가격을 올리게 하여, 결국 소비자들이 더 많은 돈을 지출하게 되는 악순환을 일으킬 수 있습니다. 인플레이션은 또한 기업들이 더 많은 수익을 내기 위해 생산을 늘리게 하여, 결국 인플레이션을 더욱 악화시키는 원인이 될 수 있습니다. 인플레이션은 또한 금융 시장에 영향을 미칠 수 있습니다. 인플레이션은 금융 시장에서 금리를 올리게 하여, 금융 기관들이 더 많은 이자를 받게 되고, 결국 소비자들이 더 많은 돈을 지출하게 되는 악순환을 일으킬 수 있습니다. 인플레이션은 또한 국가의 경제 성장률을 낮추고, 국가의 경쟁력을 떨어뜨릴 수 있습니다. 인플레이션은

dpo_output_2_cpk_30 model result ------


Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.05s/it, Generation Speed: 54.82 toks/s]


 인플레이션은 경제에 여러 가지 영향을 미칩니다. 인플레이션은 물가 상승을 의미하며, 이는 소비자들이 구매할 수 있는 물건의 양을 줄이는 결과를 낳습니다. 이는 소비자들이 더 많은 돈을 지출해야 하기 때문에 소비가 감소하게 됩니다. 이는 생산자들에게도 영향을 미칩니다. 생산자들은 더 많은 수익을 원하기 때문에 가격을 올리게 되고, 이는 소비자들에게 더 많은 물가를 부담하게 됩니다. 인플레이션은 또한 금융 시장에도 영향을 미칩니다. 인플레이션은 금리 상승을 유발할 수 있으며, 이는 금융 시장의 투자자들에게 부담을 주게 됩니다. 인플레이션은 또한 국가의 경제 성장률을 낮추고, 실업률을 높이는 결과를 낳을 수 있습니다. 인플레이션은 경제에 미치는 영향은 다양하며, 이를 관리하기 위해서는 적절한 정책이 필요합니다.    

dpo_output_2_cpk_40 model result ------


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.21s/it, Generation Speed: 45.59 toks/s]

 인플레이션은 경제에 미치는 영향은 물가 상승으로 인해 소비자들의 구매력이 떨어지고, 기업의 생산비용이 증가하여 기업의 이윤이 감소하는 등 경제에 부정적인 영향을 미친다.



In [ ]:
for idx, question in enumerate(questions):
  print(f"Question_{idx}: {question}")
  print("Default model result -----")
  print(generate_text(raw_input = question, lora_mode = "default"))
  for mode in ["dpo_output_1_cpk_40", "dpo_output_2_cpk_40"]:
    print(f"{mode} model result ------")
    print(generate_text(raw_input = question, lora_mode = mode))
    print()

Question_0: List three ways to reduce plastic waste in daily life.
Default model result -----


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.30it/s, Generation Speed: 60.17 toks/s]


 - Reduce the use of single-use plastic items such as straws, bags, and utensils.
     - Use reusable containers and bags for shopping and carrying food.
     - Recycle plastic items properly and avoid littering.
dpo_output_1_cpk_40 model result ------


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.18s/it, Generation Speed: 45.88 toks/s]


 1. Reduce the use of single-use plastic items such as straws, plastic bags, and disposable cutlery.
     2. Use reusable containers and bags for food and shopping.
     3. Recycle plastic items properly and avoid littering.

dpo_output_2_cpk_40 model result ------


...

 인플레이션은 경제에서 중요한 요소 중 하나로, 통화량의 증가로 인해 물가가 상승하는 현상을 말합니다. 인플레이션은 경제의 성장과 관련이 있으며, 경제의 성장률이 높을수록 인플레이션이 높아질 수 있습니다. 인플레이션은 물가의 상승으로 인해 소비자들의 구매력과 소득이 감소하고, 기업의 이윤이 감소하는 등 경제에 부정적인 영향을 미칠 수 있습니다. 또한 인플레이션은 금리 인상으로 이어질 수 있으며, 이는 경제의 성장률을 낮추고 실업률을 높일 수 있습니다. 따라서 인플레이션은 경제의 안정성과 성장을 위협할 수 있는 요소로, 적절한 통화정책과 경제 정책을 통해 관리해야 합니다.    
dpo_output_1_cpk_40 model result ------


Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.28s/it, Generation Speed: 54.92 toks/s]


 인플레이션은 경제에서 발생하는 물가 상승을 의미합니다. 인플레이션은 경제의 여러 가지 측면에 영향을 미칠 수 있습니다. 인플레이션은 소비자들이 물건과 서비스를 더 많이 구매할 수 있게 해주지만, 동시에 가계 부담을 증가시킬 수 있습니다. 인플레이션은 기업의 이윤을 증가시키고, 투자와 생산을 촉진할 수 있지만, 동시에 가계 소비를 억제하고, 경제 성장을 저해할 수 있습니다. 인플레이션은 통화량의 증가, 물가 상승, 그리고 경제의 성장률과 실업률에 영향을 미칠 수 있습니다. 인플레이션은 경제의 여러 가지 측면을 고려하여 적절한 정책을 취하는 것이 중요합니다.

dpo_output_2_cpk_40 model result ------


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.21s/it, Generation Speed: 47.10 toks/s]

 인플레이션은 경제에 미치는 영향은 물가 상승과 함께 경제의 성장률이 낮아지는 것과 함께, 실업률이 증가하고, 소비자들의 구매력이 떨어지는 등 여러 가지 부정적인 영향을 미칩니다.



In [ ]:
for idx, question in enumerate(questions):
  print(f"Question_{idx}: {question}")
  print("Default model result -----")
  print(generate_text(raw_input = question, lora_mode = "default"))
  for mode in ["dpo_output_1_cpk_30", "dpo_output_2_cpk_30"]:
    print(f"{mode} model result ------")
    print(generate_text(raw_input = question, lora_mode = mode))
    print()

Question_0: List three ways to reduce plastic waste in daily life.
Default model result -----


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.31it/s, Generation Speed: 60.21 toks/s]


 - Reduce the use of single-use plastic items such as straws, bags, and utensils.
     - Use reusable containers and bags for shopping and carrying food.
     - Recycle plastic items properly and avoid littering.
dpo_output_1_cpk_30 model result ------


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.93s/it, Generation Speed: 50.90 toks/s]


 1. Reduce the use of single-use plastic items such as straws, plastic bags, and disposable cutlery.
     2. Use reusable containers, bags, and water bottles instead of single-use plastic ones.
     3. Support businesses that use eco-friendly packaging and products.
     4. Recycle plastic items properly and avoid littering.
     5. Educate others about the impact of plastic waste and encourage them to make changes in their daily lives.

dpo_output_2_cpk_30 model result ------


...

 인플레이션은 경제에서 중요한 역할을 합니다. 인플레이션은 물가 상승을 의미하며, 이는 경제에서 생산과 소비를 조절하는 데 중요한 역할을 합니다. 인플레이션은 경제의 총 생산량을 증가시키고, 소비자들의 구매력을 증가시켜 경제 성장을 촉진할 수 있습니다. 그러나 너무 높은 인플레이션은 경제에 부정적인 영향을 미칠 수 있습니다. 높은 인플레이션은 물가 상승을 초래하고, 소비자들의 구매력을 떨어뜨려 경제 성장을 저해할 수 있습니다. 또한, 높은 인플레이션은 금융 시장의 불안을 유발하고, 투자자들의 신뢰를 떨어뜨릴 수 있습니다. 따라서, 정부는 적절한 정책을 통해 인플레이션을 관리하고, 경제의 안정적인 성장을 촉진하는 것이 중요합니다.
dpo_output_1_cpk_30 model result ------


Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.23s/it, Generation Speed: 54.52 toks/s]


 인플레이션은 경제에서 중요한 요소 중 하나로, 통화량의 증가로 인해 물가가 상승하는 현상을 말합니다. 인플레이션은 경제의 성장과 관련이 있으며, 높은 인플레이션은 경제 성장에 부정적인 영향을 미칠 수 있습니다. 인플레이션은 물가 상승으로 인해 소비자들이 구매력이 떨어지게 되고, 기업들은 생산비용이 증가하여 이윤을 줄이게 됩니다. 또한, 높은 인플레이션은 금리 인상으로 이어질 수 있으며, 이는 경제 성장에 부정적인 영향을 미칠 수 있습니다. 인플레이션을 관리하기 위해 중앙은행은 통화량을 조절하고, 금리를 조절하여 경제의 안정성을 유지하기 위해 노력합니다.

dpo_output_2_cpk_30 model result ------


Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.54s/it, Generation Speed: 52.79 toks/s]

 인플레이션은 경제에서 물가 상승을 의미합니다. 인플레이션은 경제의 생산과 소비를 증가시키고, 경제의 성장을 촉진할 수 있습니다. 그러나 인플레이션은 너무 높은 수준이 되면, 경제의 안정성과 성장을 저해할 수 있습니다. 인플레이션은 물가 상승과 함께, 통화량의 증가, 물가의 불안정성, 소비자들의 구매력의 감소, 기업의 생산비용 증가, 정부의 재정적자 증가, 경제의 불균형을 초래할 수 있습니다.



In [ ]:
generate_text(questions[0])



Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, Generation Speed: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.38s/it, Generation Speed: 61.09 toks/s]


' - Reduce the use of single-use plastic items such as straws, plastic bags, and disposable cutlery.\n     - Use reusable containers and bags for food and shopping.\n     - Support businesses that use sustainable packaging and reduce plastic waste.\n     - Recycle plastic items properly and avoid littering.\n     - Educate others about the impact of plastic waste and encourage them to make changes in their daily lives.    '

In [ ]:
generate_text(raw_input = questions[0], lora_mode = "dpo_output_1_cpk_10")



Processed prompts:   0%|          | 0/2 [00:00<?, ?it/s, Generation Speed: 0.00 toks/s]

Processed prompts: 100%|██████████| 2/2 [00:10<00:00,  5.04s/it, Generation Speed: 10.82 toks/s]


' 1. Reduce the use of single-use plastic items such as straws, plastic bags, and disposable cutlery.\n     2. Use reusable containers and bags for shopping and carrying food.\n     3. Recycle plastic items properly and avoid littering.'

In [ ]:
!ls /content/drive/MyDrive/dpo_output_1/checkpoint-10

adapter_config.json	   README.md	  special_tokens_map.json  trainer_state.json
adapter_model.safetensors  rng_state.pth  tokenizer_config.json    training_args.bin
optimizer.pt		   scheduler.pt   tokenizer.json


In [ ]:
answers_dict = {
  "base_model_answers": [],
  "fine_tuned_model_answers": [],
  "dpo_model_1_answers": [],
  "dpo_model_2_answers": [],
  "dpo_model_1_cpk_30_answers": [],
  "dpo_model_2_cpk_30_answers": []
}
for idx, question in enumerate(questions):
    print(f"Processing EXAMPLE {idx}==============")

    answers_dict['base_model_answers'].append(test_model(question, base_model))
    answers_dict['fine_tuned_model_answers'].append(test_model(question, fine_tuned_model))
    answers_dict['dpo_model_1_answers'].append(test_model(question, dpo_model_1))
    answers_dict['dpo_model_2_answers'].append(test_model(question, dpo_model_2))
    answers_dict['dpo_model_1_cpk_30_answers'].append(test_model(question, dpo_model_1_cpk_30))
    answers_dict['dpo_model_2_cpk_30_answers'].append(test_model(question, dpo_model_2_cpk_30))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processing EXAMPLE 0==============


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processing EXAMPLE 1==============


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processing EXAMPLE 14==============


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processing EXAMPLE 15==============


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
def simple_format(text, width=120):
    return '\n'.join(line[i:i+width] for line in text.split('\n') for i in range(0, len(line), width))


for idx, question in enumerate(questions):
    print(f"EXAMPLE {idx}==============")
    print(f"Question: {question}")

    """
    print("<<Model 1 답변>>")
    base_model_answer = answers_dict["base_model_answers"][idx].split("### Response:")[1]
    print(simple_format(base_model_answer))
    print("---")
    print("<<Model 2 답변>>")
    fine_tuned_model_answer = answers_dict["fine_tuned_model_answers"][idx].split("### Response:")[1]
    print(simple_format(fine_tuned_model_answer))
    print()
    print("---")
    print("<<Model 3 답변>>")
    dpo_model_1_answer = answers_dict["dpo_model_1_answers"][idx].split("### Response:")[1]
    print(simple_format(dpo_model_1_answer))
    print()
    print("---")
    print("<<Model 4 답변>>")
    dpo_model_2_answer = answers_dict["dpo_model_2_answers"][idx].split("### Response:")[1]
    print(simple_format(dpo_model_2_answer))
    print()
    print("---")
    print("<<Model 5 답변>>")
    dpo_model_2_answer = answers_dict["dpo_model_1_cpk_30_answers"][idx].split("### Response:")[1]
    print(simple_format(dpo_model_2_answer))
    print()
    print("---")
    print("<<Model 6 답변>>")
    dpo_model_2_answer = answers_dict["dpo_model_2_cpk_30_answers"][idx].split("### Response:")[1]
    print(simple_format(dpo_model_2_answer))
    print()
    """
    print("<<Model 2 답변>>")
    fine_tuned_model_answer = answers_dict["fine_tuned_model_answers"][idx].split("### Response:")[1]
    print(simple_format(fine_tuned_model_answer))
    print()
    print("---")
    print("<<Model 5 답변>>")
    dpo_model_2_answer = answers_dict["dpo_model_1_cpk_30_answers"][idx].split("### Response:")[1]
    print(simple_format(dpo_model_2_answer))
    print()
    print("---")
    print("<<Model 6 답변>>")
    dpo_model_2_answer = answers_dict["dpo_model_2_cpk_30_answers"][idx].split("### Response:")[1]
    print(simple_format(dpo_model_2_answer))
    print()